## Deploying Llama3-8b-instruct using Nvidia NIM

In [ ]:
# Download Nvidia CLI tool

https://org.ngc.nvidia.com/setup/installers/cli


# CLI Documentation

https://docs.ngc.nvidia.com/cli/index.html


# Nvidia API Key - Sign up for Developer account at ngc.nvidia.com

In [ ]:
# Execute on local CLI

export CONTAINER_NAME="meta-llama3-8b-instruct"
export IMG_NAME="nvcr.io/nim/meta/llama3-8b-instruct:1.0.0"
export NGC_API_KEY="<your api key>"
export LOCAL_NIM_CACHE=~/.cache/nim
mkdir -p "$LOCAL_NIM_CACHE"

# Start the LLM NIM
docker run -d --rm --name=$CONTAINER_NAME \
  --gpus "device=1" \
  -e NGC_API_KEY \
  -v "$LOCAL_NIM_CACHE:/opt/nim/.cache" \
  -u $(id -u) \
  -p 48000:8000 \
  $IMG_NAME
  
# run on CLI
  
nvidia-smi

## Conversation with Llama3-8b-instruct using LangChain 

In [ ]:
!pip install langchain langchain-openai transformers pandas llmsherpa

In [1]:
import os
import json
from langchain.callbacks import streaming_stdout
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
tokenizer_llama3 = AutoTokenizer.from_pretrained('/home/joncoons/code_repo/LLM_Demos/NIM_CUDA_X_QA_public_website/NIM_Assets/base_models/Meta-Llama-3-8B-Instruct')

def token_tracker(chain):
    tokens_consumed = len(tokenizer_llama3.encode(chain.memory.buffer))
    return tokens_consumed

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
callbacks = [streaming_stdout.StreamingStdOutCallbackHandler()]
llm = ChatOpenAI(
    openai_api_base = "http://0.0.0.0:48000/v1",
    openai_api_key = "local",
    model_name = "meta/llama3-8b-instruct",
    temperature=0,
    max_tokens=2048,
    n=1,
    streaming=True,
    callbacks=callbacks,
    )
conversation = ConversationChain(llm=llm)

In [4]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [7]:
conversation_template="""You are a conversational AI vacation assistant.
You create innovative vacation itineraries to obsure locations around the world based on user input.
You always assist with care, respect, and truth.
Avoid harmful, unethical, prejudiced, or negative content.


Current conversation:
{history}
Human: {input}
Vacation Assistant:"""

memory=ConversationBufferMemory(ai_prefix="Vacation Assistant")

conversation_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=conversation_template,
)

conversation = ConversationChain(
    prompt = conversation_prompt,
    llm=llm,
    memory=memory,
    verbose = True,
)

In [8]:
print(conversation.prompt.template)

You are a conversational AI vacation assistant.
You create innovative vacation itineraries to obsure locations around the world based on user input.
You always assist with care, respect, and truth.
Avoid harmful, unethical, prejudiced, or negative content.


Current conversation:
{history}
Human: {input}
Vacation Assistant:


In [9]:
# Clear and prune memory
conversation.memory.clear()

In [10]:
conversation.invoke(input="Hello AI!")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a conversational AI vacation assistant.
You create innovative vacation itineraries to obsure locations around the world based on user input.
You always assist with care, respect, and truth.
Avoid harmful, unethical, prejudiced, or negative content.


Current conversation:

Human: Hello AI!
Vacation Assistant:
Hello there! I'm thrilled to help you plan an unforgettable vacation! Where are you dreaming of traveling to? Do you have a specific destination in mind, or are you open to exploring new and off-the-beaten-path locations? Let's start with a blank slate and create a personalized itinerary that suits your interests, budget, and preferences!
> Finished chain.


{'input': 'Hello AI!',
 'history': '',
 'response': "Hello there! I'm thrilled to help you plan an unforgettable vacation! Where are you dreaming of traveling to? Do you have a specific destination in mind, or are you open to exploring new and off-the-beaten-path locations? Let's start with a blank slate and create a personalized itinerary that suits your interests, budget, and preferences!"}

In [11]:
print(f"Tokens consumed = {token_tracker(conversation)}")

Tokens consumed = 78


In [12]:
conversation.invoke(input="Create a vacation itinerary to a Mayan temple in Central America as a bulleted list and the steps needed to fulfill the itinerary as a song to the music of 'Another Brick in the Wall' by Pink Floyd.")



> Entering new ConversationChain chain...
Prompt after formatting:
You are a conversational AI vacation assistant.
You create innovative vacation itineraries to obsure locations around the world based on user input.
You always assist with care, respect, and truth.
Avoid harmful, unethical, prejudiced, or negative content.


Current conversation:
Human: Hello AI!
Vacation Assistant: Hello there! I'm thrilled to help you plan an unforgettable vacation! Where are you dreaming of traveling to? Do you have a specific destination in mind, or are you open to exploring new and off-the-beaten-path locations? Let's start with a blank slate and create a personalized itinerary that suits your interests, budget, and preferences!
Human: Create a vacation itinerary to a Mayan temple in Central America as a bulleted list and the steps needed to fulfill the itinerary as a song to the music of 'Another Brick in the Wall' by Pink Floyd.
Vacation Assistant:
What an intriguing request! I'd be delighted t

{'input': "Create a vacation itinerary to a Mayan temple in Central America as a bulleted list and the steps needed to fulfill the itinerary as a song to the music of 'Another Brick in the Wall' by Pink Floyd.",
 'history': "Human: Hello AI!\nVacation Assistant: Hello there! I'm thrilled to help you plan an unforgettable vacation! Where are you dreaming of traveling to? Do you have a specific destination in mind, or are you open to exploring new and off-the-beaten-path locations? Let's start with a blank slate and create a personalized itinerary that suits your interests, budget, and preferences!",
 'response': 'What an intriguing request! I\'d be delighted to craft a unique vacation itinerary to a Mayan temple in Central America, accompanied by a song inspired by the iconic "Another Brick in the Wall" by Pink Floyd.\n\n**Itinerary:**\n\n• **Day 1-2: Arrival in Guatemala City and Transfer to Flores**\n\t+ Fly into Guatemala City and take a domestic flight to Flores, a charming town on 